<a href="https://colab.research.google.com/github/soltiste/ii-in-economi/blob/main/Lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.
2. Проведите предобработку:

• Приведите тексты к нижнему регистру.

• Замените все, кроме букв и цифр, на пробелы — это облегчит
дальнейшее разделение текста на слова. Для такой замены в
строке text подходит следующий вызов:
re.sub(’[^a−zA−Z0−9]’, ’␣’, text.lower())

• Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

• Замените пропуски в столбцах LocationNormalized и ContractTime
на специальную строку ’nan’.

• Примените DictVectorizer для получения one-hot-кодирования
признаков LocationNormalized и ContractTime.

• Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и
категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.
4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.
Если ответом является нецелое число, то целую и дробную часть
необходимо разграничивать точкой, например, 0.42. При необходимости
округляйте дробную часть до двух знаков.
Ответ на каждое задание — текстовый файл, содержащий ответ в
первой строчке. Обратите внимание, что отправляемые файлы не должны содержать пустую строку в конце.

In [59]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.

In [60]:
df = pd.read_csv('/content/drive/MyDrive/salary-train.csv')
df.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


2. Проведите предобработку:

• Приведите тексты к нижнему регистру.

• Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub(’[^a−zA−Z0−9]’, ’␣’, text)

• Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

• Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку ’nan’.

• Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

• Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [61]:
df['FullDescription'] = df['FullDescription'].str.lower().str.replace('[^a-zA-Z0-9]', ' ', regex=True)

In [62]:
df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)

In [63]:
dv = DictVectorizer()
encoded = dv.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [64]:
tfidf = TfidfVectorizer(min_df=5)
tfidf_matrix = tfidf.fit_transform(df['FullDescription'])

In [65]:
X_train = hstack([tfidf_matrix, encoded])
Y_train = df['SalaryNormalized']

3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized

In [66]:
clf = Ridge(alpha=1)
clf.fit(X_train, Y_train)

Ridge(alpha=1)

4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел. Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков. Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать пустую строку в конце.

In [67]:
df_test = pd.read_csv('/content/drive/MyDrive/salary-test-mini.csv')
df_test.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [68]:
df_test['FullDescription'] = df_test['FullDescription'].str.lower().str.replace('[^a-zA-Z0-9]', ' ', regex=True)

In [69]:
encoded_test = dv.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
tfidf_matrix_test = tfidf.transform(df_test['FullDescription'])
X_test = hstack([tfidf_matrix_test, encoded_test])

In [70]:
Y_pred = clf.predict(X_test)
result = pd.Series(Y_pred)
result

0    56573.539043
1    37196.822112
dtype: float64